# Effectuer un classement des établissements selon leur niveau de bruit dans le référentiel Structure

objectif : produire un tableau avec les colonnes suivantes : nom de portail, nb structure, % de bruit

- obtenir un tableau de tous les portails
- réduire aux portails institutionnels
- trouver les structId des portails identifiés
    - explorer l'API AureHal Structure pour avoir l'id des établissements à partir d'une requête par nom
    - créer une fonction qui donne à partir du nom de l'établissement leur id
    - ajouter au tableau des établissements une colonne avec leur id
- pour chaque portail identifié extraire nb structure valide, non valide, % bruit 
    - créer une fonction qui donne : nb structure valide et non valide à partir d'un structId
    - ajouter les champs nb_struct_valid et nb_struct_no_valid au tableau des établissements
- calculer le pourcetange de bruit et l'intégrer au tableau
- retirer les etab où nous n'avons pas de structure valide, trier par noise en décroissant

## Obtenir la liste des portails HAL

In [1]:
import pandas as pd, json, requests

In [2]:
# la doc du referentiel des portails : https://api.archives-ouvertes.fr/docs/ref/resource/instance

In [3]:
# récupérer les infos de tous les portails
raw = requests.get("https://api.archives-ouvertes.fr/ref/instance").json()

In [4]:
# les placer dans une dataframe
portails = pd.DataFrame(raw['response']['docs'])
portails.sample(3)

,id,code,name,url
24,27,insu,INSU - Institut national des sciences de l'Uni...,https://hal-insu.archives-ouvertes.fr
98,105,clermont-univ,Université Clermont Auvergne,https://hal.uca.fr
176,8449,anr,Portail HAL-ANR,https://hal-anr.archives-ouvertes.fr


In [28]:
print(f"nb de portails {len(portails)}")

nb de portails 181


## Réduire aux protails institutionnels

In [29]:
pd.set_option('display.max_rows', None) #pour afficher toutes les rows de la df

In [5]:
# sélectionner les portails des institutions
filtre_name = portails["name"].str.contains(
    "univ|ecole|école|inst|inria|inserm|labo|cea|supelec|agro|cnam|enssib|ENS|CIrad|Observatoire|INSA|MNHN|inshea|lyon",
case = False, regex=True)
# test de filtre sur le champs URL
filtre_url = portails["url"].str.contains("univ|cnrs|sciencespo", case = False, regex=True)
#portails[filtre_name].sort_values(by = "name")
portail_etab = portails[filtre_name].sort_values(by = "name")
portail_etab.sample(10)

,id,code,name,url
157,7267,enssib,ENSSIB,https://hal-enssib.archives-ouvertes.fr
133,5511,univ-rochelle,Université de La Rochelle,https://hal-univ-rochelle.archives-ouvertes.fr
92,99,icp,ICP - Institut Catholique de Paris,https://hal-icp.archives-ouvertes.fr
35,38,univ-brest,Université de Bretagne Occidentale,https://hal.univ-brest.fr
61,67,agroparistech,AgroParisTech,https://hal-agroparistech.archives-ouvertes.fr
141,5853,upf,Université de la Polynésie française,https://hal-upf.archives-ouvertes.fr
79,45,univ-diderot,Université Paris Diderot-Paris 7,https://hal-univ-diderot.archives-ouvertes.fr
115,4090,univ-lorraine,Université de Lorraine,https://hal.univ-lorraine.fr
178,8822,insa-lyon,INSA Lyon,https://hal-insa-lyon.archives-ouvertes.fr
113,4025,univ-paris-dauphine,Université Paris Dauphine - Paris IX,https://hal-univ-paris-dauphine.archives-ouver...


## Trouver les `structId` des portails identifiés

In [31]:
# documentation référentiel structure https://api.archives-ouvertes.fr/docs/ref/?resource=structure&schema=fields#fields

In [7]:
def get_structId_from_name(name) : 
    """extrait le structId du nom de portail spécifié en argument """
    # recherche dans acronyme et name
    # utiliser name_t car insensible à la casse et accents
    # memo : https://api.archives-ouvertes.fr/ref/structure/?fq=type_s:institution&q=(name_s:cirad OR acronym_s:Cirad)
    
    url = "https://api.archives-ouvertes.fr/ref/structure/?fq=type_s:institution&fq=valid_s:VALID"
    req = requests.get(url + f"&q=(name_t:\"{name}\" OR acronym_t:\"{name}\" )")
    r = req.json()
    try : 
        return r["response"]["docs"][0]["docid"]
    except : 
        print("pb ", name)

In [8]:
get_structId_from_name("Université Clermont Auvergne")

pb  Université Clermont Auvergne


In [9]:
portail_etab["structId"] = portail_etab.apply(lambda row : get_structId_from_name(row["name"]), axis = 1)

pb  CEA - Commissariat à l’énergie atomique
pb  Cnam - Conservatoire national des arts et métiers
pb  DUMAS - Dépôt Universitaire de Mémoires Après Soutenance
pb  EM-LYON Business School
pb  ENAC - Ecole Nationale de l'Aviation Civile
pb  ENS de Lyon
pb  EPHE - École pratique des hautes études
pb  Ecole Centrale Paris
pb  Groupe des Écoles Nationales d'Économie et Statistiques
pb  Hal Lumière Lyon2
pb  Haut Conseil de l’évaluation de la recherche et de l’enseignement supérieur
pb  ICP - Institut Catholique de Paris
pb  IN2P3 - Institut national de physique nucléaire et de physique des particules
pb  INERIS - Institut National de l’EnviRonnement Industriel et des RisqueS
pb  INSU - Institut national des sciences de l'Univers
pb  IRD - Institut de recherche pour le développement
pb  IRSN - Institut de Radioprotection et de Sûreté Nucléaire
pb  Inalco - Institut National des Langues et Civilisations Orientales
pb  Inrap - Institut national de recherches archéologiques préventives
pb  Inse

In [14]:
portail_etab.sample(5)

,id,code,name,url,structId
121,4638,insep,"Insep - Institut national du sport, de l'exper...",https://hal-insep.archives-ouvertes.fr,NaN
17,20,cirad,Cirad,http://hal.cirad.fr,11574.0
48,54,riip,RIIP - Réseau International des Instituts Pasteur,https://hal-riip.archives-ouvertes.fr,NaN
131,5408,hceres,Haut Conseil de l’évaluation de la recherche e...,https://hal-hceres.archives-ouvertes.fr,NaN
18,21,pasteur,Institut Pasteur,https://hal-pasteur.archives-ouvertes.fr,55578.0


In [15]:
# calculer taux de détection de structId de porail

In [16]:
print( f"Des structId ont été trouvés pour { len( portail_etab[ portail_etab['structId'].notna()] ) / len(portail_etab) } des étab.") 

Des structId ont été trouvés pour 0.5615384615384615 des étab.


## Créer une fonction pour récupérer nb structure valide et non valide à partir d'un `structId`

In [17]:
def get_struct_nb(structId, valid):
    """nombre de sous structures attachées à la structure parente"""
    # si l'établissement n'a pas de structId alors on ne le traite pas
    if not pd.isna(structId) :  
        url = "https://api.archives-ouvertes.fr/ref/structure/?"
        req = requests.get(url+ f"&q=parentDocid_i:{int(structId)}&fq=valid_s:{valid}")
        req = req.json()
        return req["response"]["numFound"]    

In [18]:
# test de la foncion
get_struct_nb(172265,"INCOMING")

28

## Ajouter les champs "nb structure valide" et "nb structure non valide" au tableau des établissements

In [19]:
portail_etab["nb_struct_valid"] = portail_etab.apply(lambda row : get_struct_nb(row['structId'], "VALID"), axis = 1)

In [20]:
portail_etab.sample(5)

,id,code,name,url,structId,nb_struct_valid
35,38,univ-brest,Université de Bretagne Occidentale,https://hal.univ-brest.fr,NaN,NaN
50,55,univ-paris13,Université Paris 13,https://hal-univ-paris13.archives-ouvertes.fr,15786.0,1.0
24,27,insu,INSU - Institut national des sciences de l'Uni...,https://hal-insu.archives-ouvertes.fr,NaN,NaN
137,5815,inshea,INSHEA,https://hal-inshea.archives-ouvertes.fr,308431.0,1.0
12,14,ens-lyon,ENS de Lyon,https://hal-ens-lyon.archives-ouvertes.fr,NaN,NaN


In [21]:
portail_etab["nb_struct_no_valid"] = portail_etab.apply(lambda row : get_struct_nb(row['structId'], "INCOMING"), axis = 1)

In [22]:
portail_etab.sample(3)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid
166,7706,mines-ales,École des Mines d'Alès,https://hal.mines-ales.fr,NaN,NaN,NaN
128,5242,univ-lemans,Université du Maine,https://hal-univ-lemans.archives-ouvertes.fr,NaN,NaN,NaN
124,4808,univ-guyane,Université de Guyane,https://hal.univ-guyane.fr,424370.0,9.0,2.0


## Calculer le pourcentage de bruit et l'intégrer au tableau

In [23]:
portail_etab["noise"] = round(portail_etab["nb_struct_no_valid"] / portail_etab["nb_struct_valid"] * 100, 0)

In [24]:
len(portail_etab)

130

In [25]:
## netoyer le tableau et afficher le classement

In [26]:
# retrait des lignes sans données pour nb_struct_valid et noise
portail_noise = portail_etab[ 
    (portail_etab["nb_struct_valid"].notna()) & 
    (portail_etab["noise"].notna()) 
]
len(portail_noise)

72

In [28]:
# conserver que les etab où structId > 0
portail_noise = portail_noise[ portail_noise["nb_struct_valid"] >0 ]
portail_noise.sample(5)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
45,51,univ-savoie,Université Savoie Mont Blanc,http://hal.univ-smb.fr,432896.0,30.0,4.0,13.0
19,22,obspm,Observatoire de Paris,https://hal-obspm.ccsd.cnrs.fr,300055.0,16.0,11.0,69.0
57,64,ensieta,ENSTA Bretagne,https://hal-ensta-bretagne.archives-ouvertes.fr,301846.0,6.0,0.0,0.0
180,8935,ulco,Université du Littoral Côte d'Opale,https://hal-ulco.archives-ouvertes.fr,301404.0,18.0,5.0,28.0
70,77,ensta,ENSTA Paris,https://hal-ensta-paris.archives-ouvertes.fr/,300065.0,6.0,1.0,17.0


In [132]:
# trier par noise décroissant
portail_noise.sort_values(by = "noise", ascending = False)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
50,55,univ-paris13,Université Paris 13,https://hal-univ-paris13.archives-ouvertes.fr,15786.0,1.0,73.0,7300.0
162,7468,enva,École Nationale vétérinaire d'Alfort,https://hal-enva.archives-ouvertes.fr,34256.0,12.0,70.0,583.0
156,7264,univ-angers,Université d'Angers,https://hal.univ-angers.fr,74911.0,56.0,131.0,234.0
43,49,univ-nantes,Université de Nantes,http://hal.univ-nantes.fr,93263.0,72.0,150.0,208.0
83,90,agrocampus-ouest,Agrocampus Ouest,https://hal-agrocampus-ouest.archives-ouvertes.fr,108028.0,24.0,50.0,208.0
64,71,hcl,Hospices Civils de Lyon,https://hal-hcl.archives-ouvertes.fr,547850.0,1.0,2.0,200.0
88,91,univ-rennes1,Université de Rennes 1,https://hal-univ-rennes1.archives-ouvertes.fr,105160.0,106.0,189.0,178.0
163,7580,u-pec,Université Paris-Est Créteil Val-de-Marne,https://hal.u-pec.fr/,420786.0,95.0,146.0,154.0
103,109,insa-rennes,INSA Rennes,https://hal-insa-rennes.archives-ouvertes.fr,117606.0,7.0,10.0,143.0
10,11,inserm,Inserm,https://www.hal.inserm.fr,303623.0,602.0,802.0,133.0


---
Allez plus loin : 
    
* améliorer le matching nom de portail -> structId
* ajouter des iterations : recherches les "structures rouges" non plus au niveau de la structure parente, mais dans toute les structures filles.